#### Large RAM is required to load the larger models. Running on GPU can optimize inference speed.

In [1]:
import sys
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess
import pandas as pd

/data/home/jielinq/miniconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import numpy as np
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from pycocoevalcap.cider import cider
from pycocoevalcap.spice import spice
from bert_score import score
import pandas as pd
import numpy as np

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [4]:
model, vis_processors, _ = load_model_and_preprocess(
    name="blip2_t5", model_type="pretrain_flant5xxl", is_eval=True, device=device
)
vis_processors.keys()

Loading checkpoint shards: 100%|██████████| 5/5 [00:05<00:00,  1.10s/it]


dict_keys(['train', 'eval'])

In [5]:
def split_words(sentence):
    return sentence.split(' ')

def Rouge(GT_caption, generated_caption): # returns list of all rouge_n needed in (precision,recall,fmeasure) for each
    res = []
    scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
    output = scorer.score(GT_caption, generated_caption)
    for r in output:
        score = output[r]
        res.append((score.precision, score.recall, score.fmeasure))
    return res[0][2], res[1][2], res[2][2]

def Bleu_4(GT_caption, generated_caption):
    GT_words = split_words(GT_caption)
    gen_words = split_words(generated_caption)
    weights = (1./4., 1./4., 1./4., 1./4.)
    return corpus_bleu([[GT_words]], [gen_words], weights)

def METEOR(GT_caption, generated_caption):
    GT_words = split_words(GT_caption)
    gen_words = split_words(generated_caption)
    return meteor_score([GT_words], gen_words)

def CIDEr(GT_caption, generated_caption):
    scorer = cider.Cider()
    return scorer.compute_score({0:[GT_caption]}, {0:[generated_caption]})[0]

def SPICE(GT_caption, generated_caption):
    scorer = spice.Spice()
    return scorer.compute_score({0:[GT_caption]}, {0:[generated_caption]})[0]

def BertScore(GT_caption, generated_caption):
    P, R, F1 = score([generated_caption], [GT_caption], lang="en", verbose=True)
    #return P.tolist()[0], R.tolist()[0], F1.tolist()[0]
    return F1.tolist()[0]

def acc_full(GT_caption, generated_caption):
    GT_caption = GT_caption.lower()
    generated_caption = generated_caption.lower()
    return int(GT_caption in generated_caption)

def acc_part(GT_caption, generated_caption):
    GT_words = split_words(GT_caption.lower())
    generated_caption = generated_caption.lower()
    
    in_generated = 0
    for word in GT_words:
        if word in generated_caption:
            in_generated += 1
            
    return in_generated / len(GT_words)

In [6]:
def answer_evaluation(GT_caption, generated_caption):
    rouge1, rouge2, rougeL = Rouge(GT_caption, generated_caption)
    bleu4 = Bleu_4(GT_caption, generated_caption)
    meteor = METEOR(GT_caption, generated_caption)
    cider= CIDEr(GT_caption, generated_caption)
    spice = SPICE(GT_caption, generated_caption)
    bertscore = BertScore(GT_caption, generated_caption)
    
    return rouge1, rouge2, rougeL, bleu4, meteor, cider, spice, bertscore

In [7]:
def get_result(image_path, question):
    

    raw_image = Image.open(image_path).convert('RGB')  
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
    generated_answer = model.generate({"image": image, "prompt": "%s"%question})
    
    return generated_answer[0]

In [8]:
old_tsv = pd.read_csv("landmark_all.tsv", sep='\t', encoding='utf-8')

df_results = pd.DataFrame(columns=['GT_answer', 'generated_answer','rouge1', 'rouge2', 'rougeL', 'bleu4', 'meteor', 'cider', 'spice', 'bertscore'])

In [9]:
for i in range(0,2001,50):
    
    print(i)
    GT_answer = old_tsv.loc[i,'answer']
    generated_answer = get_result(old_tsv.loc[i,'image_path'], old_tsv.loc[i,'question'])
    rouge1_res, rouge2_res, rougeL_res, bleu4_res, meteor_res, cider_res, spice_res, bertscore_res = answer_evaluation(GT_answer, generated_answer)
    
    df_results = df_results.append({'GT_answer':GT_answer, 'generated_answer': generated_answer, 'rouge1':rouge1_res, 'rouge2':rouge2_res,'rougeL':rougeL_res, \
                                        'bleu4':bleu4_res, 'meteor':meteor_res, \
                                        'cider':cider_res, 'spice':spice_res, 'bertscore':bertscore_res}, ignore_index=True)
    
df_results.to_csv('blip_landmark.tsv', sep='\t', encoding='utf-8', index= False)

0
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Parsing reference captions
Parsing test captions


SPICE evaluation took: 767.3 ms
calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00, 64.96it/s]


computing greedy matching.


...

100%|██████████| 1/1 [00:00<00:00, 65.39it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 639.86it/s]

done in 0.02 seconds, 49.07 sentences/sec



/tmp/ipykernel_1069409/2076904643.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'GT_answer':GT_answer, 'generated_answer': generated_answer, 'rouge1':rouge1_res, 'rouge2':rouge2_res,'rougeL':rougeL_res, \
